In [1]:
import pickle

dictName = f'../../data/T2Dcounts.pkl'
dataMap = None

with open(dictName, 'rb') as f:
    dataMap = pickle.load(f)
    
print(list(dataMap.keys()))
print('Complete')

['T2D', 'NGT', 'T2D_001', 'NGT_003', 'T2D_002', 'NGT_010', 'T2D_006', 'T2D_007', 'NGT_016', 'T2D_009', 'NGT_018', 'T2D_011', 'NGT_020', 'T2D_012', 'NGT_021', 'T2D_017', 'NGT_022', 'T2D_019', 'NGT_023', 'T2D_024', 'NGT_027', 'T2D_026', 'NGT_028', 'T2D_030', 'NGT_033', 'T2D_031', 'NGT_050', 'T2D_035', 'NGT_052', 'T2D_101', 'NGT_123', 'T2D_104', 'NGT_124', 'T2D_107', 'T2D_108', 'NGT_127', 'T2D_109', 'T2D_112', 'NGT_129', 'T2D_119']
Complete


In [2]:
print(dataMap['T2D'])
print(dataMap['NGT'])
print(list(dataMap['T2D_001'].keys()))

{'T2D_017', 'T2D_012', 'T2D_024', 'T2D_026', 'T2D_112', 'T2D_019', 'T2D_011', 'T2D_104', 'T2D_030', 'T2D_002', 'T2D_007', 'T2D_006', 'T2D_031', 'T2D_109', 'T2D_119', 'T2D_009', 'T2D_001', 'T2D_107', 'T2D_108', 'T2D_101', 'T2D_035'}
{'NGT_010', 'NGT_052', 'NGT_129', 'NGT_023', 'NGT_033', 'NGT_016', 'NGT_050', 'NGT_028', 'NGT_021', 'NGT_020', 'NGT_123', 'NGT_124', 'NGT_022', 'NGT_003', 'NGT_018', 'NGT_127', 'NGT_027'}
['basal', 'post', 'rec']


In [3]:
countName = '../../data/GSE202295_gene_counts.txt'

import re
import numpy as np

pat = re.compile('".*" "(.*)" "(.*)" (.*)')

def parseCountsLine(line, pat):
    res = pat.search(line)
    try:
        groups = res.groups()
        counts = [int(c) for c in groups[2].split(' ')]
        return (groups[0], groups[1], counts)
    except Exception as e:
        return None
    
def parseCountsFirstLine(line):
    parts = line.split(' ')
    pat = re.compile('"([^A]+).*"')
    countsMap = {}
    for i in range(2,len(parts)):
        sampleId = pat.search(parts[i]).groups()[0]
        countsMap[sampleId] = i-2
    return countsMap
    
countsAll = []
countNames1 = []
countNames2 = []
countsMap = None

with open(countName, 'r') as f:
    for line in f.readlines():
        res = parseCountsLine(line, pat)
        if res is None:
            countsMap = parseCountsFirstLine(line)
            continue
        n1,n2,counts = res
        countNames1.append(n1)
        countNames2.append(n2)
        countsAll.append(np.array(counts))
        
countsAll = np.stack(countsAll)
print(countsAll.shape)
print(countNames2[0:10])
print('Complete')

(66023, 111)
['RP11-399E6.2', 'AC093151.1', 'FO681542.1', 'RP11-399E6.4', 'DPPA2P2', 'RPS6KA1', 'MIR1976', 'RN7SL679P', 'Y_RNA', 'RNU7-29P']
Complete


In [65]:
import re

imp = []
with open('../../data/omics-regions/post_Corr.txt', 'r') as f:
    for line in f.readlines():
        imp = [int(num) for num in re.split('\s+', line) if len(num) > 0]
        
imp = np.array(imp)
print(imp)

[43567 26941 52161 59236 23705  3014 12102  8093 33518 18699 60217 29246
 26520 11845 47688  8623 35479 25801  2369 18803]


In [21]:
import random
import numpy as np

def getSamples(dataMap, timep):
    ngt = []
    t2d = []
    for sid in dataMap['NGT']:
        if timep in dataMap[sid]:
            ngt.append(dataMap[sid][timep])
    for sid in dataMap['T2D']:
        if timep in dataMap[sid]:
            t2d.append(dataMap[sid][timep])
    return np.stack(ngt), np.stack(t2d)

from sklearn.linear_model import LogisticRegression
import sys

sys.path.append('../..')

from imagenomer.main import Analysis, JsonData

nRuns = 50
C = 1000
max_iter = 5000
timep = 'post'
modelName = 'Logistic Regression'

model = f'{modelName} L2 C={C} max_iter={max_iter}'
desc = f'T2D({timep}) vs NGT({timep}) exercise muscle biopsy genecounts Wonly'
analysis = Analysis(desc=f'{desc}: {model}', 
                    host='localhost', 
                    port=8000, 
                    label_names=['Indices', 'Gene IDs'],
                    labels=[list(range(countsAll.shape[0])), countNames2])

tot = []

for run in range(nRuns):
    ngt, t2d = getSamples(dataMap, timep)
    np.random.shuffle(ngt)
    np.random.shuffle(t2d)
    a = int(3*ngt.shape[0]/4)
    b = int(3*t2d.shape[0]/4)
    c = ngt.shape[0]-a
    d = t2d.shape[0]-b
    Xtrain = np.concatenate([ngt[:a], t2d[:b]])
    Xtest = np.concatenate([ngt[a:], t2d[b:]])
    ytrain = np.concatenate([np.zeros(a), np.ones(b)])
    ytest = np.concatenate([np.zeros(c), np.ones(d)])
    
    Xtrain /= np.max(Xtrain, axis=1, keepdims=True)
    Xtest /= np.max(Xtest, axis=1, keepdims=True)
    
    clf = LogisticRegression(max_iter=5000, C=C).fit(Xtrain, ytrain)
    yhat = clf.predict(Xtest)
    acc = sum(yhat == ytest)/len(ytest)
    
    print(f'{run}. {acc}')
    
    tot.append(acc)
    
    # Raw weights
    w = list(clf.coef_.squeeze())
    
    # Send Weights
    jsonObj = {
        'Compare': desc,
        'Model': modelName,
        'Accuracy': float(acc),
        'Train': [a,b],
        'Test': [c,d],
        'Weights': list(w)
    }
    
    dat = JsonData(analysis)
    dat.update(jsonObj)
    r = dat.post()
    print(dat.dict['runid'])
    print(r.content)
    
tot = np.array(tot)
print(np.mean(tot))
print(np.std(tot))

0. 0.6666666666666666
0
b'Success'
1. 0.5555555555555556
1
b'Success'
2. 0.8888888888888888
2
b'Success'
3. 0.7777777777777778
3
b'Success'
4. 0.5555555555555556
4
b'Success'
5. 0.3333333333333333
5
b'Success'
6. 0.4444444444444444
6
b'Success'
7. 0.5555555555555556
7
b'Success'
8. 0.5555555555555556
8
b'Success'
9. 0.6666666666666666
9
b'Success'
10. 0.6666666666666666
10
b'Success'
11. 0.5555555555555556
11
b'Success'
12. 0.5555555555555556
12
b'Success'
13. 0.6666666666666666
13
b'Success'
14. 0.5555555555555556
14
b'Success'
15. 0.5555555555555556
15
b'Success'
16. 0.5555555555555556
16
b'Success'
17. 0.7777777777777778
17
b'Success'
18. 0.4444444444444444
18
b'Success'
19. 0.5555555555555556
19
b'Success'
20. 0.7777777777777778
20
b'Success'
21. 0.4444444444444444
21
b'Success'
22. 0.6666666666666666
22
b'Success'
23. 0.5555555555555556
23
b'Success'
24. 0.6666666666666666
24
b'Success'
25. 0.6666666666666666
25
b'Success'
26. 0.4444444444444444
26
b'Success'
27. 0.666666666666666

In [26]:
(a,b,c,d)

(12, 15, 4, 5)

In [70]:
ngt, t2d = getSamples(dataMap, timep)
ngt = np.stack(ngt)
t2d = np.stack(t2d)

X = np.concatenate([ngt, t2d])
X = X.T

y = np.concatenate([np.ones((1,len(ngt))), -np.ones((1,len(t2d)))], axis=1)
y = np.repeat(y, 66023, axis=0)

print(y.shape)
print(X.shape)

(66023, 36)
(66023, 36)
